# Sheet 5 Exercise 3: Nonlinear Neural Field (Credits: 4)
Note: All analytical computations are supposed to be carried out by hand! You need to submit your computations to complete the exercise. You can either write your computations into the notebook using LaTeX syntax, or submit them on paper (scan).

Consider a nonlinear neural field of Amari type with the step threshold function. The model is given by the equation
$$
\tau\dot{u}(x,t) = -u(x,t) + \int_{-\infty}^{\infty} w(x-x^\prime)1(u(x^\prime,t))\text{d}x^\prime + s(x,t) + h.
$$

Assume that the resting potential is given by $h=-1$. The input $s$ and the interaction kernel $w$ are given by piecewise-linear functions, namely
$$
s(x) := \begin{cases} C\left( 1-\frac{\vert x\vert}{d} \right), & \vert x\vert \leq d \\ 0, & \text{elsewhere}, \end{cases}
$$
and
$$
w(x) := \begin{cases} r, & \vert x\vert < a \\ -s, & a\leq\vert x\vert\leq b \\ 0, & \text{elsewhere}. \end{cases}
$$

#### 3.1 (mandatory)

Simulate the equation to approximately find the time-dependent solution of the model. You can do this exactly as in Exercise 2 using the Euler method and a Riemann-sum-approximation of the integral. Use the parameters $\tau=1$, $r=3$, $s=1.75$, $C=0.6$, $a=1$, $b=3$, $d=4$ and $h=-1$. For the spatial domain, use $A=-20$, $B=20$ as the integral borders.

First, start the model at resting level (i.e. $u(x,0) = h$). Then use the initial condition $u(x,0) = 3.3 s(x) + h$. Finally, use the interaction kernel $\hat{w}(x) := 0.1 w(x)$ and the initial condition $u(x,0) = 3.3 s(x) + h$.

Create 3D plots of the three time-dependent solutions. What do you oberserve about the stationary solutions, and the evolution towards them? Can you explain the results?

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### YOUR CODE HERE ###

default_args = {
                'A': -20,   # start of the x interval
                'B': 20,    # stop of the x interval
                'n': 300,   # number of x-points
                'tau': 1,  # time-constant of the system
                'a': 1,     # interaction param
                'b': 3,   # interaction param
                'd': 4,     # input param
                'C': 0.6,    # 
                'dt': 1e-2, # temporal resolution of Euler's method
                'T': 2,    # time to stop simulation
                'add_noise': False,
                'sigma': 0.01 # noise sqrt(variance)
               }    


def get_interaction_kernel(x_grid, args):
    # setting params
    a, b, k0 = args['a'], args['b'], args['k0']
    n = len(x_grid)
    dx = x_grid[1] - x_grid[0]
    
    W = np.zeros((n, n))
    for i in range(n):
        for j in range(n):   
            delta_x = x_grid[i] - x_grid[j]
            W[i, j] = a / np.sqrt(np.pi * b**2) * (np.exp(-(delta_x**2 / (4 * b**2))) * np.cos(k0 * delta_x)) * dx
            
    return W


def get_input(x_grid, args):
    """Return **vector** (n, 1) of input"""
    d = args['d']
    s = 1 / (2 * np.sqrt(np.pi * d**2)) * np.exp(-x_grid**2 / (4 * d**2))
    if args['add_noise']:
        sigma = args['sigma']
        noise = np.random.normal(scale=sigma, size=len(x_grid))
        s += noise
    return s.reshape((len(x_grid), 1))


def get_du(u, W, s, args):
    """
    Input:
        u: np.array with shape (n, 1)
        W: (n, n) interaction kernel
        s: (n, 1) constant input
        args: configuration for the simulation. 
        
        Parameters needed to be specified: args['tau'], args['dt']
        
    Returns:
        du: (n, 1) vector to update the solution on the next timestep
    """
    dt = args['dt']
    tau = args['tau']
    
    # why there is matrix multiplication see the notes. 
    # It is a different notation for calculating Riemann integral for our special case
    du = dt / tau * (-u + W @ u + s)
    return du


def run_simulation(args, verbose=True):
    
    """
    Runs finite difference estimation for the neural fields specified by "args".
    Returns t, x, u:
        t: np.array, len(np.arange(0, T, dt))
        x: np.array, len(np.linspace(A, B, n))
        u: np.array, shape = (len(t), len(x))
    """
    
    if verbose: print(args)
    
    n = args['n']
    
    t_grid = np.arange(0, args['T'], args['dt'])
    x_grid = np.linspace(args['A'], args['B'], args['n'])
    
    s = get_input(x_grid, args)              # (n, 1)
    W = get_interaction_kernel(x_grid, args) # (n, n)
    
    # initialize solution array
    # will get this done with lists cause of nasty vectors of shape (n x 1)
    u_list = [np.zeros((n, 1))] # also set initial value u(x, t(0))
    
    for t in t_grid:
        du = get_du(u_list[-1], W, s, args)
        u_list.append(u_list[-1] + du)

    return t_grid, x_grid, np.array(u_list).squeeze()


def plot_simulation(t, x, u, args):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    ax = axes[0]
    ax.imshow(u.T, aspect='auto', extent=[t[0], t[-1], x[0], x[-1]], cmap='gray')
    ax.set_xlabel("Time ($t$)")
    ax.set_ylabel("Space ($x$)")
    ax.set_title("Solution $u(x, t)$ \n" + f"Params: b = {args['b']}, d = {args['d']}, k0 = {args['k0']}")
    ax.grid()

    ax = axes[1]
    ax.plot(x, u[-1, :], color='black', linewidth=2)
    ax.set_title("Steady state solution $u(x, T)$")
    ax.set_xlabel("Coordinate $x$")

### YOUR CODE HERE ###

**Explanation:** 

#### 3.2 (Non-mandatory)
Analytically compute the stationary solution for the case in which there is no excited region in the field. Why does no peak arise here? What is the threshold value for the input amplitude $C$ that is needed for an activated region to emerge?

**Explanation:** 

#### 3.3 (mandatory)
Consider again the first interaction kernel $w$ and the initial condition $u(x,0) = 3.3 s(x) + h$. Now set the input signal to zero (but not the initial condition!). Simulate the model. Which stationary solution emerges? How do you explain the result?

**Explanation:** 

#### 3.4 (Non-mandatory)
In the situation of 3.3, analytically compute the exact size of the stable peak. 

*Hint:* Use the theorems found by Amari (1977). You can check your results using the plot from 3.3.